Goal: start training neural network. get better than random accuracy.

In [51]:
import torch

In [52]:
import torchvision.datasets as datasets
from torchvision import transforms

transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_data = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

In [53]:
from torch.utils.data import DataLoader
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)


I guess I need to make an actual model here.

In [54]:
import torch.nn as nn
import torch.optim as optim

class Skynet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.hidden = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [55]:
input_size = 28 ** 2
hidden_size = 16
output_size = 10

model = Skynet(input_size, hidden_size, output_size)


In [56]:
loss_func = torch.nn.CrossEntropyLoss()

dummy_preds = torch.rand(4, 10)
dummy_labels = torch.tensor([1, 2, 9, 0])

loss = loss_func(dummy_preds, dummy_labels)

In [57]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [58]:
def train_one_epoch():
    for features, labels in train_loader:
        optimizer.zero_grad()
        preds = model(features)
        loss = loss_func(preds, labels)
        loss.backward()
        optimizer.step()

In [59]:
train_one_epoch()

In [60]:
# model.train(True)
# for _ in range(0, 10):
#     train_one_epoch()

In [61]:
model.train(False)
preds = []
for features, labels in test_loader:
    with torch.no_grad():
        batch_preds = model(features)
        preds.extend(batch_preds.tolist())

In [62]:
preds_tensor = torch.tensor(preds)
category_preds = torch.argmax(preds_tensor, dim=1)

In [63]:
def check_accuracy(preds, actual):
    if len(actual) != len(preds):
        return -1
    return sum([int(actual[i] == preds[i]) for i in range(0, len(actual))]) / len(actual)

In [64]:
print(f"Accuracy: {check_accuracy(category_preds, test_data.targets)}")

Accuracy: 0.8512


MUAHAHAHA, out of nowhere I suddenly have 83% accuracy with only one training epoch. With only one hidden layer, one epoch, no recurrency, and tons of other potential tweaks, I have a ton of room to further improve. Before I do that though I want to get cuda set up, will save plenty of time once I get to more epochs and layers.